In [1]:
import Pkg

package_list = ["DataFrames", "CSV",
 "JuMP", "Gurobi",
 "LinearAlgebra", "Random", "Printf", "StatsBase", "CategoricalArrays",
 "Plots", "StatsPlots",
 "Distributions", "JLD"]

for pkg_name in package_list
    Pkg.update(pkg_name)
    Pkg.add(pkg_name)
end

    Updating registry at `~/.julia/registries/General.toml`
   Installed GR_jll ────────────── v0.69.1+0
   Installed Measures ──────────── v0.3.2
   Installed InlineStrings ─────── v1.3.2
   Installed StatsFuns ─────────── v1.1.0
   Installed LoggingExtras ─────── v1.0.0
   Installed OffsetArrays ──────── v1.12.8
   Installed MutableArithmetics ── v1.1.0
   Installed EarCut_jll ────────── v2.2.4+0
   Installed RecipesPipeline ───── v0.6.11
   Installed OpenSSL ───────────── v1.3.2
   Installed NearestNeighbors ──── v0.4.12
   Installed StaticArrays ──────── v1.5.11
   Installed InvertedIndices ───── v1.2.0
   Installed HTTP ──────────────── v1.5.5
   Installed FileIO ────────────── v1.16.0
   Installed Qt5Base_jll ───────── v5.15.3+2
   Installed SentinelArrays ────── v1.3.16
   Installed PlotThemes ────────── v3.1.0
   Installed StaticArraysCore ──── v1.4.0
   Installed BenchmarkTools ────── v1.3.2
   Installed TranscodingStreams ── v0.9.10
   Installed DataFrames ────────── v1.4.4
 

In [2]:
using DataFrames, CSV
using JLD
using JuMP, Gurobi
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots
using Distributions

┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1662
┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1662
┌ Info: Precompiling Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1662
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1662
┌ Info: Precompiling StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1662


In [3]:
liverpool_data = CSV.read("Liverpool_FIFA_22.csv", DataFrame)
n_players = nrow(liverpool_data)

33

In [4]:
all_teams = CSV.read("Team_Ratings.csv", DataFrame)
n_positions = 27
C = CSV.read("fixture_list.csv", DataFrame)
n_games = nrow(C)

38

In [5]:
GK = "gk"
defenders = ["lwb", "rwb", "lb", "lcb", "cb", "rcb", "rb"]
midfielders = ["lam", "cam", "ram", "lm", "lcm", "cm", "rcm", "rm", "ldm", "cdm", "rdm"]
forwards = ["ls", "st", "rs", "lw", "lf", "cf", "rf", "rw"]
center_backs = ["lcb", "cb", "rcb"]
right_backs = ["rwb", "rb"]
left_backs = ["lwb", "lb"]
cdms = ["ldm", "cdm", "rdm"]
cms = ["lcm", "cm", "rcm"]
cams = ["lam", "cam", "ram"]
left_wingers = ["lw", "lm"]
right_wingers = ["rw", "rm"]
strikers = ["ls", "st", "rs", "lf", "cf", "rf"]

6-element Vector{String}:
 "ls"
 "st"
 "rs"
 "lf"
 "cf"
 "rf"

In [6]:
player_pos = ["ls", "st", "rs", "lw", "lf", "cf", "rf", "rw", "lam", "cam", "ram", "lm", "lcm", "cm", "rcm", "rm", "lwb", "ldm", "cdm", "rdm", "rwb", "lb", "lcb", "cb", "rcb", "rb", "gk"]
pos_to_ind = Dict()
index = 1
for pos in player_pos
    pos_to_ind[pos] = index
    index += 1
end
pos_to_ind["st"]

2

In [7]:
R = zeros(Int64, (n_players, n_positions))
for i in 1:n_players
    for pos in player_pos
        R[i, pos_to_ind[pos]] = liverpool_data[i, pos]
    end
end
R

33×27 Matrix{Int64}:
 73  73  73  67  69  69  69  67  73  …  87  87  82  84  89  89  89  84  23
 89  89  89  87  88  88  88  87  89     68  68  71  68  62  62  62  68  22
 88  88  88  88  88  88  88  88  89     71  71  74  70  61  61  61  70  25
 40  40  40  35  38  38  38  35  43     38  38  34  33  34  34  34  33  90
 75  75  75  79  77  77  77  79  82     85  85  88  87  82  82  82  87  21
 78  78  78  80  79  79  79  80  84  …  86  86  87  86  81  81  81  86  22
 77  77  77  81  80  80  80  81  86     83  83  80  78  74  74  74  78  21
 77  77  77  75  75  75  75  75  79     87  87  85  85  87  87  87  85  21
 84  84  84  83  83  83  83  83  85     75  75  75  72  70  70  70  72  20
 79  79  79  78  78  78  78  78  82     84  84  83  82  82  82  82  82  22
 64  64  64  62  64  64  64  62  68  …  82  82  76  78  83  83  83  78  21
 84  84  84  82  82  82  82  82  83     71  71  71  70  67  67  67  70  20
 60  60  60  64  62  62  62  64  66     80  80  80  81  83  83  83  81  19
  ⋮ 

In [45]:
# model = Model(with_optimizer(Gurobi.Optimizer))
model = Model(Gurobi.Optimizer)

### Decision Variables ###

### x[i][j][k]. player i, game j, position k ###
@variable(model, x[i=1:n_players, j=1:n_games, k=1:n_positions], Bin)

### z variable, for whether we go for win or not ###
@variable(model, z[j = 1:n_games], Bin)


### Match Winning Constraint ###
### R is player ratings, R[i][k] is player i for position k ###
### C is the opposing team rating, C[j] is the rating of the opposing team for game j ###

for j in 1:n_games
    @constraint(model, sum(sum(R[i, k]*x[i, j, k] for i in 1:n_players) for k in 1:n_positions)/11 >= C[j, 2]*z[j])
end

### Player Constraints ###

### Each player can only play at most once in a game ###
for j in 1:n_games
    for i in 1:n_players
        @constraint(model, sum(x[i, j, k] for k in 1:n_positions) <= 1)
    end
end

### Can only have 11 players in game ###
for j in 1:n_games
    @constraint(model, sum(sum(x[i, j, k] for k in 1:n_positions) for i in 1:n_players) == 11)

    ### 1 GK per game ###
    @constraint(model, sum(x[i, j, pos_to_ind[GK]] for i in 1:n_players) == 1)

    ### 4 Defenders per game ###
    for k in defenders
        @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 4)
    end

    ### 3 - 5 Midfielders per game ### 
    for k in midfielders
        # @constraint(model, 3 <= sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 5)
        @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 5)
        # @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) >= 0)
    end

    ### 1 - 3 Forwards per game ###
    for k in forwards
        # @constraint(model, 1 <= sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 3)
        @constraint(model, 1 <= sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 3)
    end

    # ### 2 CBs per game ###
    # for k in center_backs
    #     @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) == 2)
    # end

    # ### 1 RB per game ###
    # for k in right_backs
    #     @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) == 1)
    # end

    # ### 1 LB per game ###
    # for k in left_backs
    #     @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) == 1)
    # end

    # ### 0 - 2 CDMs per game ###
    # for k in cdms
    #     @constraint(model, 0 <= sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 2)
    # end

    # ### 0 - 3 CMs per game ###
    # for k in cms
    #     @constraint(model, 0 <= sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 3)
    # end

    # ### 0 - 3 CAMS per game ###
    # for k in cams
    #     @constraint(model, 0 <= sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 3)
    # end

    # ### 0 - 1 LW, LM per game ###
    # for k in left_wingers
    #     @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 1)
    # end

    # ### 0 - 1 RW, RM per game ###
    # for k in right_wingers
    #     @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) <= 1)
    # end

    # ### 1 ST per game ###
    # for k in strikers
    #     @constraint(model, sum(x[i, j, pos_to_ind[k]] for i in 1:n_players) == 1)
    # end 
end

### Objective Function ###
@objective(model, Max, sum(z[j] for j in 1:n_games))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-03


z[1] + z[2] + z[3] + z[4] + z[5] + z[6] + z[7] + z[8] + z[9] + z[10] + z[11] + z[12] + z[13] + z[14] + z[15] + z[16] + z[17] + z[18] + z[19] + z[20] + z[21] + z[22] + z[23] + z[24] + z[25] + z[26] + z[27] + z[28] + z[29] + z[30] + z[31] + z[32] + z[33] + z[34] + z[35] + z[36] + z[37] + z[38]

In [47]:
optimize!(model)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2356 rows, 34200 columns and 135774 nonzeros
Model fingerprint: 0xafabba93
Variable types: 304 continuous, 33896 integer (33896 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e+00, 1e+01]
Presolved: 44 rows, 339 columns, 1006 nonzeros

Continuing optimization...


Explored 1 nodes (30 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 3: 36 35 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.600000000000e+01, best bound 3.600000000000e+01, gap 0.0000%

User-callback calls 24, time in user-callback 0.00 sec


In [51]:
opt_x = value.(x)
for i in 1:n_players
    for k in 1:n_positions
        cur_val = opt_x[i, 1, k]
        if cur_val == 1
            println(i, " ", k)
        end
    end
end

1 24
2 7
3 8
4 27
7 6
8 1
12 3
17 2
18 8
29 4
31 5
